In [14]:
import pandas as pd

df = pd.read_csv('escooter_data.csv')
df = df.dropna(subset=['Start Community Area Number', 'End Community Area Number'])
df

,Trip ID,Start Time,End Time,Trip Distance,Trip Duration,Accuracy,Start Census Tract,End Census Tract,Start Community Area Number,End Community Area Number,Start Community Area Name,End Community Area Name,Start Centroid Latitude,Start Centroid Longitude,Start Centroid Location,End Centroid Latitude,End Centroid Longitude,End Centroid Location
10291,0dd95046-1bc8-4464-ba39-7f9e92c668b3,08/06/2019 06:00:00 PM,08/06/2019 06:00:00 PM,1004,236,152,1.703120e+10,1.703120e+10,20.0,20.0,HERMOSA,HERMOSA,41.935048,-87.737604,POINT (-87.7376041954734 41.9350481285613),41.935048,-87.737604,POINT (-87.7376041954734 41.9350481285613)
10293,6819df6a-9623-4ae0-b1d3-ec617197d229,07/08/2019 04:00:00 PM,07/08/2019 05:00:00 PM,9124,3182,10,1.703115e+10,1.703119e+10,15.0,19.0,PORTAGE PARK,BELMONT CRAGIN,41.949592,-87.761958,POINT (-87.7619580301513 41.9495915671753),41.920143,-87.760827,POINT (-87.7608269556143 41.920143229102)
10342,aa71bb53-3e7f-4964-b6e6-030800419346,08/10/2019 09:00:00 PM,08/10/2019 09:00:00 PM,502,97,152,1.703184e+10,1.703184e+10,27.0,27.0,EAST GARFIELD PARK,EAST GARFIELD PARK,41.885045,-87.713862,POINT (-87.71386248059811 41.8850448415253),41.885045,-87.713862,POINT (-87.71386248059811 41.8850448415253)
10367,5d256770-1a8a-5e20-8f78-316d1819f46b,06/25/2019 07:00:00 PM,06/25/2019 08:00:00 PM,5053,2167,0,1.703122e+10,1.703119e+10,22.0,19.0,LOGAN SQUARE,BELMONT CRAGIN,41.921126,-87.699754,POINT (-87.6997540616386 41.9211256768265),41.927819,-87.751362,POINT (-87.7513623675995 41.9278186319205)
10369,dffe5324-5f6f-48e7-9edf-d51876ba7d2f,09/19/2019 08:00:00 AM,09/19/2019 08:00:00 AM,790,111,152,1.703119e+10,1.703120e+10,19.0,20.0,BELMONT CRAGIN,HERMOSA,41.935218,-87.744389,POINT (-87.7443888733072 41.9352176386575),41.927852,-87.735628,POINT (-87.7356282729632 41.9278522690551)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710834,536f855f-2530-4c07-b93e-4de52e75cf1b,09/06/2019 07:00:00 AM,09/06/2019 07:00:00 AM,2524,500,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710835,88721836-1a6c-4ae6-b320-8e9ce1861d16,07/11/2019 11:00:00 AM,07/11/2019 11:00:00 AM,2475,480,10,NaN,NaN,20.0,19.0,HERMOSA,BELMONT CRAGIN,41.924348,-87.734740,POINT (-87.7347401601846 41.9243477570064),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710836,3f930717-06cf-499d-b390-a45282851347,09/17/2019 05:00:00 PM,09/17/2019 05:00:00 PM,2228,590,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710837,11d01331-0a12-607e-11d0-13310a12607e,06/21/2019 05:00:00 PM,06/21/2019 05:00:00 PM,1303,1088,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)


In [15]:
df.columns

Index(['Trip ID', 'Start Time', 'End Time', 'Trip Distance', 'Trip Duration',
       'Accuracy', 'Start Census Tract', 'End Census Tract',
       'Start Community Area Number', 'End Community Area Number',
       'Start Community Area Name', 'End Community Area Name',
       'Start Centroid Latitude', 'Start Centroid Longitude',
       'Start Centroid Location', 'End Centroid Latitude',
       'End Centroid Longitude', 'End Centroid Location'],
      dtype='object')

In [21]:
datetime_format = "%m/%d/%Y %I:%M:%S %p"
import pandas as pd
import numpy as np

start_areas = df['Start Community Area Number'].unique()
end_areas = df['End Community Area Number'].unique()
all_regions = np.unique(np.concatenate((start_areas, end_areas)))

df['Start Time'] = pd.to_datetime(df['Start Time'], format=datetime_format)
df['End Time'] = pd.to_datetime(df['End Time'], format=datetime_format)

df['Start Hour'] = df['Start Time'].dt.hour
df['End Hour'] = df['End Time'].dt.hour
df['Day'] = (df['Start Time'] - df['Start Time'].min()).dt.days

trips_per_hour = df.groupby(['Day', 'Start Hour', 'Start Community Area Number']).size().reset_index(name='requests')

trips_per_hour['predicted_requests'] = trips_per_hour.groupby(['Day', 'Start Community Area Number'])['requests'].shift(-1)
trips_per_hour['predicted_requests'] = trips_per_hour.groupby(['Day', 'Start Community Area Number'])['predicted_requests'].fillna(0)

outflow = df.groupby(['Start Community Area Number', df['Start Time'].dt.date]).size().reset_index(name='outflow')
inflow = df.groupby(['End Community Area Number', df['End Time'].dt.date]).size().reset_index(name='inflow')
net_flow = pd.merge(outflow, inflow, left_on=['Start Community Area Number', 'Start Time'], right_on=['End Community Area Number', 'End Time'])
net_flow['net_outflow'] = net_flow['outflow'] - net_flow['inflow']
max_outflow = net_flow.groupby('Start Community Area Number')['net_outflow'].max().reset_index()

trips_per_hour = trips_per_hour.merge(max_outflow, on='Start Community Area Number', how='left')
trips_per_hour.rename(columns={'net_outflow': 'initial_available_scooters'}, inplace=True)

trips_per_hour['available_scooters'] = trips_per_hour['initial_available_scooters']

all_days = df['Day'].unique()
all_hours = range(24)
index = pd.MultiIndex.from_product([all_days, all_hours, all_regions], names=['Day', 'Start Hour', 'Start Community Area Number'])
complete_df = pd.DataFrame(index=index).reset_index()

final_df = complete_df.merge(trips_per_hour, on=['Day', 'Start Hour', 'Start Community Area Number'], how='left')

final_df['requests'].fillna(0, inplace=True)
final_df['predicted_requests'].fillna(0, inplace=True)
final_df

,Day,Start Hour,Start Community Area Number,requests,predicted_requests,initial_available_scooters,available_scooters
0,52,0,2.0,0.0,0.0,NaN,NaN
1,52,0,3.0,0.0,0.0,NaN,NaN
2,52,0,4.0,0.0,0.0,NaN,NaN
3,52,0,5.0,0.0,0.0,NaN,NaN
4,52,0,6.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
108235,121,23,60.0,0.0,0.0,NaN,NaN
108236,121,23,61.0,0.0,0.0,NaN,NaN
108237,121,23,63.0,0.0,0.0,NaN,NaN
108238,121,23,66.0,0.0,0.0,NaN,NaN


In [22]:
# Drop the 'initial_available_scooters' column
final_df.drop('initial_available_scooters', axis=1, inplace=True)

# Set 'available_scooters' to 5 if it's smaller than 0 or NaN
final_df['available_scooters'] = final_df['available_scooters'].apply(lambda x: 5 if x < 0 or pd.isna(x) else x)
final_df

,Day,Start Hour,Start Community Area Number,requests,predicted_requests,available_scooters
0,52,0,2.0,0.0,0.0,5.0
1,52,0,3.0,0.0,0.0,5.0
2,52,0,4.0,0.0,0.0,5.0
3,52,0,5.0,0.0,0.0,5.0
4,52,0,6.0,0.0,0.0,5.0
...,...,...,...,...,...,...
108235,121,23,60.0,0.0,0.0,5.0
108236,121,23,61.0,0.0,0.0,5.0
108237,121,23,63.0,0.0,0.0,5.0
108238,121,23,66.0,0.0,0.0,5.0


In [23]:
final_df.to_csv('experiment_data.csv', index=None)